In [1]:
%load_ext autoreload
%autoreload 2
import startup_credit as cr
RAW_CSV      = r"C:\Users\Romain\OneDrive - KU Leuven\Masters\MBIS\Year 2\Semester 2\Statistical Consulting\ExternalData\startup_failures.csv"         # your Crunchbase-style file
OUT_DIR= r"C:\Users\Romain\OneDrive - KU Leuven\Masters\MBIS\Year 2\Semester 2\Statistical Consulting\ExternalData" # where to save the output

c:\Users\Romain\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
cr.train(RAW_CSV,OUT_DIR,None)

c:\Users\Romain\OneDrive - KU Leuven\Masters\MBIS\Year 2\Semester 2\Statistical Consulting\startup_credit.py:609: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  out = pd.to_datetime(val, errors="coerce", utc=True)
[I 2025-05-01 14:08:19,369] A new study created in memory with name: no-name-4673823d-2c32-4551-bdb6-4ec99c73a991
[I 2025-05-01 14:13:07,549] Trial 0 finished with value: 0.7616495865837651 and parameters: {'n_estimators': 600, 'learning_rate': 0.17254716573280354, 'max_depth': 6, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746}. Best is trial 0 with value: 0.7616495865837651.
[I 2025-05-01 14:15:52,574] Trial 1 finished with value: 0.7868159346134236 and parameters: {'n_estimators': 400, 'learning_rate': 0.011900590783184251, 'max_depth': 7, 'subsample': 0.8404460046972835, 'colsample_bytree': 0.8832290311184181}. Best is trial 1 with va

In [2]:
cr.score(r"C:\Users\Romain\OneDrive - KU Leuven\Masters\MBIS\Year 2\Semester 2\Statistical Consulting\results_xgb\results\xgb_model.joblib",RAW_CSV,r"C:\Users\Romain\OneDrive - KU Leuven\Masters\MBIS\Year 2\Semester 2\Statistical Consulting\ExternalData\out.csv")

ModuleNotFoundError: No module named 'Romain'

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# --------------------------------------------------
# Raw counts from the table
# --------------------------------------------------
exit_states = ["Acquired", "Closed", "IPO", "Operating"]
grades      = ["R1", "R2", "R3", "R4", "R5"]

xgb_counts = {
    "Acquired": [1141, 1914, 416, 658, 1420],
    "Closed":   [406, 1051, 283, 606, 3892],
    "IPO":      [892,  490,  47,  58,  60],
    "Operating":[28924,14340,1789,2708,5273],
}

rf_counts = {
    "Acquired": [987,  2517, 742, 467, 836],
    "Closed":   [285,  1370, 529, 542, 3512],
    "IPO":      [786,  632,  59,  38,  32],
    "Operating":[28007,16885,2790,1642,3710],
}

# --------------------------------------------------
# Helper: convert category counts to percentages
# --------------------------------------------------
def to_perc(counts):
    total = sum(counts)
    return [c * 100 / total for c in counts]

xgb_perc = [to_perc(xgb_counts[e]) for e in exit_states]
rf_perc  = [to_perc(rf_counts[e])  for e in exit_states]

# --------------------------------------------------
# Plotting parameters
# --------------------------------------------------
x      = np.arange(len(exit_states))  # category positions
width  = 0.30                         # bar width
gap    = 0.05                         # horizontal gap between XGB and RF stacks

colors = ["#bdd7e7", "#6baed6", "#3182bd", "#08519c", "#08306b"]  # light→dark = R1→R5

fig, ax = plt.subplots(figsize=(10, 6))

bottom_xgb = np.zeros(len(exit_states))
bottom_rf  = np.zeros(len(exit_states))

# --------------------------------------------------
# Build stacked bars
# --------------------------------------------------
for i, grade in enumerate(grades):
    xgb_vals = [xgb_perc[j][i] for j in range(len(exit_states))]
    rf_vals  = [rf_perc[j][i]  for j in range(len(exit_states))]

    # XGBoost (left stack of each pair)
    ax.bar(x - width/2 - gap/2, xgb_vals, width,
           bottom=bottom_xgb,
           label=grade if i == 0 else "",
           color=colors[i])

    # Random Forest (right stack of each pair)
    ax.bar(x + width/2 + gap/2, rf_vals,  width,
           bottom=bottom_rf,
           color=colors[i])

    bottom_xgb += xgb_vals
    bottom_rf  += rf_vals

# --------------------------------------------------
# Formatting
# --------------------------------------------------
ax.set_ylabel("Percentage share within exit state (%)")
ax.set_title("Rating distribution by exit state — XGBoost (left) vs Random Forest (right)")
ax.set_xticks(x)
ax.set_xticklabels(exit_states)
ax.set_ylim(0, 105)

# Single-grade legend
handles = [plt.Rectangle((0, 0), 1, 1, color=colors[i]) for i in range(len(grades))]
ax.legend(handles, grades, title="Rating grade",
          bbox_to_anchor=(1.05, 1), loc="upper left")

plt.tight_layout()
plt.savefig("exit_state_distribution.png", dpi=300, bbox_inches="tight")
plt.show()
